In [10]:
import boto3
from sagemaker.model import Model
from sagemaker import get_execution_role, Session

role = get_execution_role()
session = Session()

ecr_image = '491085388405.dkr.ecr.us-east-1.amazonaws.com/fraudulent-ecr-23:latest'

model = Model(
    image_uri=ecr_image,
    role=role,
    sagemaker_session=session
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name='fraudulent-ml-endpoint'
)


----!

In [14]:
import boto3
import json

# Create SageMaker runtime client
client = boto3.client('sagemaker-runtime', region_name='us-east-1')  # update region if needed

# Sample input — should match model's expected feature shape and order
payload = {
    "inputs": [[27.0, 1.0, 500987.0, 820870.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0]]
}

# Invoke the deployed endpoint
response = client.invoke_endpoint(
    EndpointName='fraudulent-ml-endpoint',  # Make sure this matches exactly
    Body=json.dumps(payload),
    ContentType='application/json'
)

# Read and decode the prediction
result = response['Body'].read().decode('utf-8')
print("Prediction:", result)


Prediction: {"predictions":[0]}



In [15]:
import boto3
import pandas as pd
import json

# Load your CSV
df = pd.read_csv('processed_data2.csv')  # Each row should have feature values

# Convert the data to list of lists (assuming features are in columns)
data = df.values.tolist()

# Initialize SageMaker client
client = boto3.client('sagemaker-runtime')

# Predict in a loop (or you can do it in batches) 
predictions = []
for row in data:
    payload = json.dumps({'inputs': [row]})
    response = client.invoke_endpoint(
        EndpointName='fraudulent-ml-endpoint',
        Body=payload,
        ContentType='application/json'
    )
    result = json.loads(response['Body'].read().decode()) 
    predictions.append(result)

# Save predictions to CSV
df['prediction_fraudulent'] = predictions
df.to_csv('predictions_fraudulent.csv', index=False)
